In [1]:
!pip install wikipedia
!pip install pyttsx3 SpeechRecognition


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6aa2e561e2d3d75041194ce7aee0af146ca229cdcbff5c5639b79d6fefd51cca
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.1 MB/s eta 0:00:00


In [2]:
!apt-get update -qq
!apt-get install -y portaudio19-dev
!pip install pyaudio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (197 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [3]:
import wikipedia
import nltk
import pyttsx3
import speech_recognition as sr
from nltk.chat.util import Chat, reflections
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag


In [5]:
import time
import pyaudio
import wave
import speech_recognition as sr


In [6]:
def record_audio(filename, duration):
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    fs = 44100

    p = pyaudio.PyAudio()

    print('錄製語音...')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []

    start_time = time.time()
    while time.time() - start_time < duration:
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    print('錄音完成.')

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [7]:
def recognize_audio(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='zh-TW')
            print("您說的是:", text)
        except sr.UnknownValueError:
            print("無法識別輸入。")


In [9]:
print("按下回車鍵開始錄音...")
input()  # 等待回車鍵

record_audio('recorded_audio.wav', duration=5)
recognize_audio('recorded_audio.wav')


按下回車鍵開始錄音...

錄製語音...


OSError: [Errno -9996] Invalid input device (no default output device)

In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
text = "I love this product! It's amazing."
sentiment_scores = sid.polarity_scores(text)

# 输出情感极性得分
print("Positive score:", sentiment_scores['pos'])
print("Negative score:", sentiment_scores['neg'])
print("Neutral score:", sentiment_scores['neu'])


Positive score: 0.734
Negative score: 0.0
Neutral score: 0.266


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [13]:
nltk.download('averaged_perceptron_tagger')
# 模式匹配
patterns = [
    (r'你好', '嗨！'),
    (r'(.*)天氣如何', '我不是氣象专家，但你可以查看天氣预报。')
]

# 分词
text = "NLTK is a powerful library for AI chat bot"
important_words = nltk.word_tokenize(text)
print(important_words)

# 句子拆分
text = """The sun rose in the clear blue sky, casting its warm rays upon the vibrant green landscape.
Birds chirped their melodious tunes as a gentle breeze rustled the leaves of the trees."""
sentences = sent_tokenize(text)
for sentence in sentences:
    print(sentence)

# 词性标注
words = word_tokenize("NLTK is a powerful library for AI chat bot")
tags = pos_tag(words)
print(tags)


['NLTK', 'is', 'a', 'powerful', 'library', 'for', 'AI', 'chat', 'bot']
The sun rose in the clear blue sky, casting its warm rays upon the vibrant green landscape.
Birds chirped their melodious tunes as a gentle breeze rustled the leaves of the trees.
[('NLTK', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('powerful', 'JJ'), ('library', 'NN'), ('for', 'IN'), ('AI', 'NNP'), ('chat', 'WP'), ('bot', 'VBD')]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
def search_wikipedia(query):
    try:
        result = wikipedia.summary(query, sentences=2)
        return result
    except wikipedia.exceptions.DisambiguationError as e:
        return "請提供更具體的查詢。"
    except wikipedia.exceptions.PageError as e:
        return "找不到相關信息。"

In [17]:
# 自定義回應規則
pairs = [
    ("你好|嗨|哈囉", ["嗨，有什麼我可以幫助你的嗎？"]),
    ("我想知道 (.*)", ["你為什麼想知道 {0}？"]),
    ("(.*)天氣如何?", ["我不是氣象專家，但你可以查看天氣預報。"]),
    ("我感覺 (.*)", ["為什麼你感覺 {0}？", "可以告訴我更多關於你的感受嗎？"]),
    ("你可以做什麼?", ["我可以回答你的問題，或者和你聊天。"]),
    ("謝謝", ["不客氣！", "很高興能幫助你！"]),
    ("再見|掰掰|拜拜", ["再見，祝你有美好的一天！"])
]

def respond(input_string, response_rules):
    for pattern, responses in response_rules:
        match = re.match(pattern, input_string)
        if match:
            response = random.choice(responses).format(*match.groups())
            return response
    return "抱歉，我不太明白你的意思。"





# 使用示例
query = "Python (programming language)"
print(search_wikipedia(query))

def chatbot():
    print("你好！我是一個簡單的聊天機器人。請輸入你的問題或對話。")
    chat = Chat(pairs, reflections)
    chat.converse()

if __name__ == "__main__":
    nltk.download("punkt")  # 下載必要的資源
    chatbot()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
你好！我是一個簡單的聊天機器人。請輸入你的問題或對話。
>你可以做什麼?
我可以回答你的問題，或者和你聊天。
>好喔
None


KeyboardInterrupt: Interrupted by user